In [32]:
import multiprocessing
import os
import pickle
import random

import gensim
import numpy as np
import pandas as pd
import sys
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
sys.path.append("..")
from embeddings_reproduction import embedding_tools

In [33]:
def get_sequence(line):
    line2=line
    for i in 'agctu\n':
        line2 = line2.replace(i, '')
    line = line.upper().rstrip()
    line = line.replace('T', 'U')
    line2 = line2.replace('T','U')
    return line2

In [34]:
sequence_dict = {}
df = pd.DataFrame(columns=['sequence'])
seq={}
all_tasks=['ALKBH5_Baltz2012',
'C17ORF85_Baltz2012',
'C22ORF28_Baltz2012']
#'CAPRIN1_Baltz2012',
#'CLIPSEQ_AGO2'
#'CLIPSEQ_ELAVL1',
#'CLIPSEQ_SFRS1',
#'ICLIP_HNRNPC',
#'ICLIP_TDP43',
#'ICLIP_TIA1',
#'ICLIP_TIAL1',
#'PARCLIP_AGO1234',
#'PARCLIP_ELAVL1',
#'PARCLIP_EWSR1',
#'PARCLIP_FUS',
#'PARCLIP_HUR',
#'PARCLIP_IGF2BP123',
#'PARCLIP_MOV10_Sievers',
#'PARCLIP_PUM2',
#'PARCLIP_QKI',
#'PARCLIP_TAF15',
#'PTBv1',
#'ZC3H7B_Baltz2012']
for task in all_tasks:
    seq={}
    print(task)
    f=open('data/'+task+'.train.positives.fa')
    for line in f:
        if line.startswith('>'):
                name=line.replace('>','').split()[0]
                seq[name]=''
        else:
                seq[name]+=line.replace('\n','').strip()
    f=open('data/'+task+'.train.negatives.fa')
    for line in f:
        if line.startswith('>'):
                name=line.replace('>','').split()[0]
                seq[name]=''
        else:
                seq[name]+=line.replace('\n','').strip()
    for name in seq.keys():
        df=df.append({'sequence':get_sequence(seq[name])},ignore_index=True)
f.close()
#print(seq)

#df.to_csv("tmp.csv")
sequence_dict['virus']=df


ALKBH5_Baltz2012
C17ORF85_Baltz2012
C22ORF28_Baltz2012


In [35]:
print(df)
df.to_csv('tmp.csv')

                                                sequence
0                              AAUAACGCUGGCGGGGAGGCCUCAG
1                    GUGAUGCCAGGACCAUCCUGGAAGAGAACAUCCAG
2                         ACAAUACAUUCCACCACAAAUACACUCUUG
3                              AUGAUGAUUGUUAAUGUUAAAGCAU
4                        UAAACAUUUUUAGUUGGAAAAACAGCAUCUG
5                          CGCCAUGCCCUUCUCCAACAGCCACAACG
6                         CCAGGCCUGCAGCAGGAGUGGUGCAGACAG
7                            AGAUUUUAGGAGCAGAAGGAUUUGUUU
8                             UUGUAUUUGUCACCUGGGGUGACAAG
9                              GAGGCCCUCUUGACUCUGCGGUGGU
10                         ACUGGGGAGAAAUCCUCUGGCUCUUCAAG
11                                GACAACCCCGCCGGGGCUGGGG
12                          CAGAGGUUCCAGGUCAUCACCAGAGCCC
13                  UUGACAUUCAGAUGAUUGUUUUUAAAUGUUUUACUU
14                             AUAGGAGGCAUUUCAAGAGGAUGCG
15                     AUGAAGCUGUCGUCAUGGCAACAGUGAGUGAAG
16                          CGG

In [36]:
def train(X, k, window):
    name_list = [X, str(k), str(window)]
    if os.path.isfile('../outputs/docvec_models/2_' + '_'.join(name_list) + '.pkl'):
        print(name_list)
        return
    print('X\t\tk\twindow')
    print(name_list[0] + '\t\t' + '\t'.join(name_list[1:]))
    kmer_hypers = {'k':k, 
                   'overlap':False,
                   'merge':False}
    model_hypers = {'size': 64,
                    'min_count': 0,
                    'iter': 25,
                    'window':window,
                    'workers': 4}
    documents = embedding_tools.Corpus(sequence_dict[X], kmer_hypers)
    model = Doc2Vec(**model_hypers)
    model.build_vocab(documents)
    model.train(documents,epochs=model.epochs,total_examples=model.corpus_count)
    model.save('outputs/docvec_models/0_' + '_'.join(name_list) + '.pkl')

train('virus',5,6)

X		k	window
virus		5	6


/Users/xiqiao/anaconda3/envs/spyder/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Users/xiqiao/anaconda3/envs/spyder/lib/python3.6/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
